In [5]:
# Return time cocnepts of a certain level inside all the vocabulary schemes in our triple store  

install.packages("SPARQL")
library(SPARQL)

Installing package into ‘/nethome/max/.R-libs’
(as ‘lib’ is unspecified)

Loading required package: XML

Loading required package: RCurl



In [6]:
# set up end point and prefix
endpoint = "http://virtuoso.nkn.uidaho.edu:8890/sparql/"
  
# attach SPARQL querry prefix. Note: the graph for our study should be updated
sparql_prefix = " 
prefix dc: <http://purl.org/dc/elements/1.1/> 
prefix dcterms: <http://purl.org/dc/terms/> 
prefix foaf: <http://xmlns.com/foaf/0.1/> 
prefix geo: <http://www.opengis.net/ont/geosparql#> 
prefix gts: <http://resource.geosciml.org/ontology/timescale/gts#> 
prefix isc: <http://resource.geosciml.org/classifier/ics/ischart/> 
prefix owl: <http://www.w3.org/2002/07/owl#> 
prefix rank: <http://resource.geosciml.org/ontology/timescale/rank/> 
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
prefix samfl: <http://def.seegrid.csiro.au/ontology/om/sam-lite#> 
prefix sf: <http://www.opengis.net/ont/sf#> 
prefix skos: <http://www.w3.org/2004/02/skos/core#> 
prefix sosa: <http://www.w3.org/ns/sosa/> 
prefix thors: <http://resource.geosciml.org/ontology/timescale/thors#> 
prefix time: <http://www.w3.org/2006/time#> 
prefix ts: <http://resource.geosciml.org/vocabulary/timescale/> 
prefix vann: <http://purl.org/vocab/vann/> 
prefix void: <http://rdfs.org/ns/void#> 
prefix xkos: <http://rdf-vocabulary.ddialliance.org/xkos#> 
prefix xsd: <http://www.w3.org/2001/XMLSchema#> 
prefix tsna: <http://deeptimekb.org/tsna#> 
prefix tssc: <http://deeptimekb.org/tssc#> 
prefix tsnc: <http://deeptimekb.org/tsnc#> 
prefix tsnz: <http://deeptimekb.org/tsnz#> 
prefix tswe: <http://deeptimekb.org/tswe#> 
prefix tsbr: <http://deeptimekb.org/tsbr#> 
prefix tsba: <http://deeptimekb.org/tsba#> 
prefix tsjp: <http://deeptimekb.org/tsjp#> 
prefix tsau: <http://deeptimekb.org/tsau#> 
        "

In [8]:
# Get a list of vocabulary scheme URIs

q0 = paste0(
              sparql_prefix, 
                  '
                   SELECT   DISTINCT  ?schURI  str(?lbl) AS ?schemeName
                   WHERE
                   {
                     GRAPH <http://deeptimekb.org/iscallnew> 
                     {
                        ?schURI a skos:ConceptScheme ;
                             rdfs:label ?lbl .
                     }      
                   }
                  '
              )

# run the query  
schemeList = SPARQL(endpoint, q0)$results
  
schListLength =length(schemeList$schURI)


In [34]:
# print out all Age level concepts in each vocabulary scheme 

for (k in 1:schListLength) {

print(schemeList$schURI[k])
print(schemeList$schemeName[k])  

# In the sparql code below we use a UNION clause to tackel the ontology patterns for 
#  version control of the international vocabulary schemes 
    
q = paste0(
              sparql_prefix, 
                  '
SELECT DISTINCT  str(?lbl) AS ?conceptName ?baseNum
WHERE
{
                
   GRAPH <http://deeptimekb.org/iscallnew>
   { 

     {
      ?concept  a gts:GeochronologicEra ,
                 gts:Age ; 
               skos:inScheme ', schemeList$schURI[k] , '; 
               rdfs:label ?lbl ;
               time:hasBeginning ?basePosition . 
     ?basePosition time:inTemporalPosition ?baseTime .
     ?baseTime time:numericPosition ?baseNum .
    }

     UNION 
     { 
       ?concept  a gts:GeochronologicEra ,
                 gts:Age ; 
               skos:inScheme ', schemeList$schURI[k] , '; 
               rdfs:label ?lbl ;
               dc:description
                   [ 
                       time:hasBeginning ?basePosition ; 
                       skos:inScheme ', schemeList$schURI[k] , '
                   ] . 
       ?basePosition time:inTemporalPosition ?baseTime .

       ?baseTime   dc:description
                   [ 
                      time:numericPosition ?baseNum ;
                      skos:inScheme ', schemeList$schURI[k] , '
                   ] .
     }     

    
  }
   
}
ORDER BY ?baseNum
                  '
              )
# run the query  
resConcepts = SPARQL(endpoint, q)$results
  
print(resConcepts)

cat("======","\n\n")
    
}

[1] "<http://resource.geosciml.org/vocabulary/timescale/tsnc2019>"
[1] "Ontology for Geologic Timescale Elements in North China"
          conceptName baseNum
1     Fengshanian Age  489.69
2    Changshanian Age  495.50
3       Kushanian Age  501.01
4       Changhian Age  503.58
5    Hsuchuangian Age  507.01
6    Maochuangian Age  509.50
7  Lungwangmaioan Age  511.75
8   Tsanglangpuan Age  514.60
9  Chiungchussuan Age  520.00
10     Jinningian Age  541.00

[1] "<http://resource.geosciml.org/vocabulary/timescale/tssc2019>"
[1] "Ontology for Geologic Timescale Elements in South China"
             conceptName baseNum
1     Penglaizhenian Age  149.93
2         Suiningian Age  163.06
3         Shaximioan Age  166.81
4       Xintiangouan Age  174.15
5       Ziliujingian Age  201.36
6           Erqiaoan Age  216.62
7      Huobachengian Age  230.51
8           Bananian Age  237.00
9          Falangian Age  241.50
10       Guanlingian Age  246.80
11   Yongningzhenian Age  249.81
12    Feixiangu